In [2]:
%pip install numpy
%pip install Pandas

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\leeco\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\leeco\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import pandas as pd

In [58]:
history = pd.concat([pd.read_csv('Results_202203.csv'), pd.read_csv('Results_202204.csv'), pd.read_csv('Results_202205.csv')])
history = history[history['ResultStatus'] == 'CompletedRace']
results = pd.read_csv('ResultsToPredict.csv')
results = results[results['ResultStatus'] == 'CompletedRace']
results.columns

Index(['RaceId', 'RaceName', 'CourseId', 'CourseName', 'RaceType', 'Class',
       'Pattern', 'RatingBand', 'AgeBand', 'SexRestriction', 'Distance',
       'DistanceInFurlongs', 'DistanceInMeters', 'DistanceInYards', 'Going',
       'Surface', 'HorseId', 'HorseName', 'JockeyId', 'JockeyName',
       'TrainerId', 'TrainerName', 'Age', 'HeadGear', 'RaceCardNumber',
       'StallNumber', 'Weight', 'WeightInPounds', 'FractionalOdds',
       'DecimalOdds', 'OfficialRating', 'RacingPostRating', 'TopSpeedRating',
       'ResultStatus', 'FinishingPosition', 'BeatenDistance',
       'OverallBeatenDistance', 'RaceTime', 'RaceTimeInSeconds'],
      dtype='object')

In [88]:
# Ensure that the majority of horses in the results to be predicted have ran races in the last 3 months of historic results
historic_racers = history.groupby(['HorseId']).size().reset_index(name='HistoryCount')
results_with_historic_race_counts = pd.merge(results, historic_racers, on='HorseId', how='left').fillna({'HistoryCount': 0})
results_with_historic_race_counts[['RaceId', 'HorseId', 'HistoryCount', 'FinishingPosition']].head(10)

winners_than_never_previously_raced = len(results_with_historic_race_counts[(results_with_historic_race_counts['HistoryCount'] == 0) & (results_with_historic_race_counts['FinishingPosition'] == 1)])
number_of_races = results_with_historic_race_counts['RaceId'].nunique()
percentage_of_winners_that_never_raced = (winners_than_never_previously_raced / number_of_races) * 100.0
(number_of_races, winners_than_never_previously_raced, percentage_of_winners_that_never_raced)

# 3 months = (number_of_races = 419, winners_than_never_previously_raced = 72, percentageOfWinnersThatNeverRaced = 17.18%)
# TODO: Add a check to ensure that the percentage of winners that never raced is less than 10%
# TODO: Increase history data so that the percentage of winners that never raced is less than 5%

(419, 72, 17.18377088305489)

In [89]:
# Calculate number of races where all runners have previously raced
def calculate_race_aggregates(x):
    d = {}
    d['HorseCount'] = x['HorseId'].count()
    d['PreviouslyRanHouseCount'] = x[x['HistoryCount'] > 0]['HorseId'].count()
    return pd.Series(d, index=['HorseCount', 'PreviouslyRanHouseCount'])

races_with_run_counts = results_with_historic_race_counts.groupby('RaceId').apply(calculate_race_aggregates)
races = len(races_with_run_counts)
races_where_all_horses_previously_ran = len(races_with_run_counts[races_with_run_counts['HorseCount'] == races_with_run_counts['PreviouslyRanHouseCount']])
races_where_80_percent_of_horses_previously_ran = len(races_with_run_counts[races_with_run_counts['PreviouslyRanHouseCount'] > races_with_run_counts['HorseCount'] * 0.8])
races_where_50_percent_of_horses_previously_ran = len(races_with_run_counts[races_with_run_counts['PreviouslyRanHouseCount'] > races_with_run_counts['HorseCount'] * 0.5])
races_where_no_horses_previously_ran = len(races_with_run_counts[races_with_run_counts['PreviouslyRanHouseCount'] == 0])

(races, races_where_all_horses_previously_ran, races_where_80_percent_of_horses_previously_ran, races_where_80_percent_of_horses_previously_ran, races_where_no_horses_previously_ran)
#(419, 127, 234, 234, 6)
# TODO: Ignore races where predictor accuracy is low (based on logic below)

(419, 127, 234, 234, 6)

In [77]:
def calculate_distance_type(row):
    if row['DistanceInMeters'] < 1300:
        return 'VeryShort'
    elif row['DistanceInMeters'] < 1700:
        return 'Short'
    elif row['DistanceInMeters'] < 3000:
        return 'Medium'
    elif row['DistanceInMeters'] < 4000:        
        return 'Long'
    else:
        return 'VeryLong'

history['DistanceType'] = history.apply(calculate_distance_type, axis=1)
results['DistanceType'] = results.apply(calculate_distance_type, axis=1)

In [80]:
def calculate_average_speed_over_previous_races(x):
    d = {}
    d['RacesRan'] = x['HorseId'].count()
    d['TotalDistanceInMeters'] = x['DistanceInMeters'].sum()
    d['TotalTimeInSeconds'] = x['RaceTimeInSeconds'].sum()
    d['AverageSpeed'] = d['TotalDistanceInMeters'] / d['TotalTimeInSeconds']
    return pd.Series(d, index=['RacesRan', 'TotalDistanceInMeters', 'TotalTimeInSeconds', 'AverageSpeed'])

average_speeds = history.groupby(['HorseId', 'HorseName', 'RaceType', 'DistanceType', 'Going']).apply(calculate_average_speed_over_previous_races)

In [85]:
average_speeds.sort_values('RacesRan')
results_with_speeds = pd.merge(results, average_speeds, how='left', on=['HorseId', 'RaceType', 'DistanceType', 'Going'])
results_with_speeds['AverageSpeed'] = results_with_speeds['AverageSpeed'].fillna(0)

1307

In [93]:
def calculate_speed_race_aggregates(x):
    d = {}
    d['HorseCount'] = x['HorseId'].count()
    d['PreviouslyRanOnSimilarCourseCount'] = x[x['AverageSpeed'] > 0]['HorseId'].count()
    return pd.Series(d, index=['HorseCount', 'PreviouslyRanOnSimilarCourseCount'])

races_with_speed_counts = results_with_speeds.groupby('RaceId').apply(calculate_speed_race_aggregates)

races = len(races_with_speed_counts)
races_where_all_horses_previously_ran = len(races_with_speed_counts[races_with_speed_counts['HorseCount'] == races_with_speed_counts['PreviouslyRanOnSimilarCourseCount']])
races_where_80_percent_of_horses_previously_ran = len(races_with_speed_counts[races_with_speed_counts['PreviouslyRanOnSimilarCourseCount'] > races_with_speed_counts['HorseCount'] * 0.8])
races_where_50_percent_of_horses_previously_ran = len(races_with_speed_counts[races_with_speed_counts['PreviouslyRanOnSimilarCourseCount'] > races_with_speed_counts['HorseCount'] * 0.5])
races_where_at_least_one_horses_previously_ran = len(races_with_speed_counts[races_with_speed_counts['PreviouslyRanOnSimilarCourseCount'] > 0])
races_where_no_horses_previously_ran = len(races_with_speed_counts[races_with_speed_counts['PreviouslyRanOnSimilarCourseCount'] == 0])

(races, races_where_all_horses_previously_ran, races_where_80_percent_of_horses_previously_ran, races_where_80_percent_of_horses_previously_ran,
 races_where_at_least_one_horses_previously_ran, races_where_no_horses_previously_ran)
# (419, 7, 21, 21, 334, 85)
# TODO: Ignore races where no horses have previously run on a similar track

(419, 7, 21, 21, 334, 85)

In [152]:
# Predict outcomes of races where at least one horse has run on a previously similar track
pd.options.display.max_rows = 500
races_where_at_least_one_horses_previously_ran = races_with_speed_counts[races_with_speed_counts['PreviouslyRanOnSimilarCourseCount'] > 0].reset_index()
race_list = races_where_at_least_one_horses_previously_ran['RaceId'].tolist()
races_to_predict = results_with_speeds[results_with_speeds['RaceId'].isin(race_list) & results_with_speeds['AverageSpeed'] > 0]

predictions = races_to_predict.sort_values(['RaceId', 'AverageSpeed'], ascending=False).groupby(['RaceId']).cumcount() + 1
races_to_predict = races_to_predict.assign(PredictedPosition=predictions)
correct_predictions = races_to_predict[(races_to_predict['FinishingPosition'] == 1) & (races_to_predict['PredictedPosition'] == 1)]

# correct_predictions[['RaceId', 'CourseId', 'HorseId', 'FinishingPosition', 'AverageSpeed', 'PredictedPosition', 'DecimalOdds', 'FractionalOdds']].head(500)
predictable_races = len(races_to_predict[races_to_predict['FinishingPosition'] == 1])
correctly_predicted = len(correct_predictions)
incorrectly_predicted = predictable_races - correctly_predicted
percentage_predicted_correctly = correctly_predicted / predictable_races * 100

# Calculate winnings and losses based on a £1 bet on each race
stake_per_race = 1
stake = predictable_races * stake_per_race
losses = incorrectly_predicted * stake_per_race
winnings = correct_predictions['DecimalOdds'].sum() * stake_per_race
profit_loss = winnings - losses
percentage_gains = profit_loss / stake 

(predictable_races, correctly_predicted, percentage_predicted_correctly, stake, winnings, losses, profit_loss, percentage_gains)

(161,
 61,
 37.88819875776397,
 161,
 391.5861111111111,
 100,
 291.5861111111111,
 1.8110938578329883)